# Analyze city property inventory

In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
DB = create_engine('postgresql://jon@localhost:5432/propertydb')

### Statistics by `parceltype`

In [3]:
propertiespt = pd.read_sql_query("""SELECT parceltype, COUNT(DISTINCT parceleproppgh.parcelid), SUM(lotarea::int) AS lotarea
FROM parceleproppgh
JOIN assessments
ON assessments.parcelid = parceleproppgh.parcelid
--WHERE parcelstatus = 'Available for Sale'
GROUP BY parceltype
ORDER BY count DESC;""", DB)

# Merge permanent city ownership parcel types
propertiespt.loc[propertiespt.parceltype.isin([
    'Greenway', 'Park', 'Infrastructure Protection', 'City Facility', 'Other Public Use'
]), 'parceltype'] = 'PCO'
propertiespt = propertiespt.groupby(propertiespt.parceltype).aggregate({ 'count': 'sum', 'lotarea': 'sum' }).sort_values(by='count')

# Convert lot area to acres
propertiespt['lotarea'] = propertiespt.lotarea / 43560

# Percent of city neighborhood acreage per parcel type
propertiespt['pct_city'] = 100 * (propertiespt.lotarea / 33161.843)

# Multiple of Schenley Park (somewhat recognizable comparison)
propertiespt['mult_schenley'] = propertiespt.lotarea / 456

propertiespt

,count,lotarea,pct_city,mult_schenley
parceltype,,,,
PLB Transfer,3,0.225367,0.000680,0.000494
Unknown Public Use,10,0.529982,0.001598,0.001162
Unknown,180,83.621671,0.252162,0.183381
CDC Property Reserve,192,11.389555,0.034345,0.024977
PCO,2240,2287.426102,6.897765,5.016285
Public Sale,2712,225.179522,0.679032,0.493815
URA Transfer,2815,200.797153,0.605507,0.440345
Hold For Study,4991,2748.636042,8.288550,6.027711


### City facilities and `Hold For Study`

In [4]:
facilities = pd.read_sql_query("""SELECT parcelfacilitiespgh.parcelname, parceltype, parcelstatus
FROM parcelfacilitiespgh
JOIN parcelcentroids
ON parcelcentroids.parcelmbl = parcelfacilitiespgh.parcelmbl
LEFT JOIN parceleproppgh
ON parceleproppgh.parcelid = parcelcentroids.parcelid;""", DB)

##### Short list of facilities

In [5]:
facilities[facilities.parceltype == 'Hold For Study'].head(10)

,parcelname,parceltype,parcelstatus
5,Firehouse 27,Hold For Study,Hold For Study
6,Brighton Heights Pool Guard Room,Hold For Study,Hold For Study
7,Brighton Heights Pool Building,Hold For Study,Hold For Study
8,Brighton Heights Park Third Base Dugout,Hold For Study,Hold For Study
9,Brighton Heights Park Storage Building,Hold For Study,Hold For Study
10,Brighton Heights Park Shelter,Hold For Study,Hold For Study
11,Brighton Heights Park First Base Dugout,Hold For Study,Hold For Study
12,Brighton Heights Field 6 Third Base Dugout,Hold For Study,Hold For Study
13,Brighton Heights Field 6 First Base Dugout,Hold For Study,Hold For Study
16,DOMI Asphalt Division Storage Building,Hold For Study,Hold For Study


##### Percent of facilities for each parcel type

In [6]:
facilitiesparceltype = facilities.parceltype.value_counts().reset_index()
facilitiesparceltype['pct'] = facilitiesparceltype['count'] / facilitiesparceltype['count'].sum()
facilitiesparceltype

,parceltype,count,pct
0,Park,194,0.500000
1,Hold For Study,186,0.479381
2,City Facility,4,0.010309
3,URA Transfer,2,0.005155
4,Greenway,1,0.002577
5,Public Sale,1,0.002577
